# Cloud orchestration

- **Estimated time**: 30 minutes
- **Requirements**:
  - An active Chameleon allocation
  - Some familiarity with the OpenStack CLI

In this tutorial you'll learn about orchestration, a powerful pattern when dealing with multiple cloud entities and the relationships between those entities. You'll follow both a simple and more complex example of orchestration, and hopefully leave with some ideas about how you might creat your own orchestration template to automate your experiment setup.

## Background: what is orchestration?

When utilizing cloud resources, it is often the case that several different components of the cloud infrastructure must be configured for your use case. For example, a new isolated network might be required, as well as a cluster of servers connected to that network. Furthermore, your cluster should be aware of its own configuration; each node should understand how to contact the others via their IP addresses. Additional configuration may be desired, such as a public IP address to allow you to remotely manage your cluster. All of these resources must be configured correctly each time you want to set up your experimental environment, which can take a significant amount of time (and be prone to mistakes.)

Cloud orchestration solves this problem by automating much of this work. With orchestration, your environment is declared in code, and then the cloud system translates the code into a set of configured resources on the cloud. Cloud orchestration solutions are typically implemented as _templates_, e.g. a domain-specific language (usually JSON or YAML-based) that defines your entities on the cloud and how they interact. The template language should afford for some way to declare dependencies between components, such that if B depends on A, then A should be created/configured first.

### Orchestration in Chameleon

We will demonstrate how to leverage orchestration in Chameleon using Heat, the OpenStack service responsible for performing the orchestration automation. Heat has some compatibility with the AWS [CloudFormation](https://aws.amazon.com/cloudformation/) tool, so if you are familiar with that, many concepts will directly apply. There are a few terms we should introduce before going further:

- **Heat**: the OpenStack service used to perform orchestration for Chameleon
- **Template**: a YAML file containing the declaration of the desired cloud resources
- **Stack**: the name of a set of resources orchestrated together: the instantiation of a template

## Tutorial

1. [A simple orchestration stack](#A-simple-orchestration-stack)
1. [Hadoop cluster orchestration](#Hadoop-cluster-orchestration)

### Variables you'll see/use in this Notebook

  - `KEYPAIR_NAME`: the name of the SSH keypair to attach to any instances launched
  - `LEASE_NAME`: the name of your lease for resources required for this tutorial
  - `OS_REGION_NAME`: the Chameleon site name, e.g. CHI@TACC or CHI@UC
  - `OS_PROJECT_NAME`: the Chameleon project to authenticate under. You might need to set this if no project was automatically chosen for you (which can happen if you are on multiple projects!)
  
### Set variables

In [ ]:
KEYPAIR_NAME="$USER-jupyter"
# Default to CHI@UC
OS_REGION_NAME="${OS_REGION_NAME:-CHI@UC}"
if [[ -z "${OS_PROJECT_NAME:+x}" ]]; then
  OS_PROJECT_NAME="CH-XXXXXX" # For example.
fi

# We will also ensure that this Key Pair exists in the site
key_pair_upload "$KEYPAIR_NAME"

### Reserve resources for this tutorial

For this orchestration tutorial, we will be reserving three bare metal nodes.

In [ ]:
LEASE_NAME="$USER-tutorial-orchestration"

node_type=compute_skylake
num_nodes=3 # We now reserve 3 nodes!

echo "Reserving $num_nodes nodes of type '$node_type' ..."

blazar lease-create \
  --physical-reservation min=$num_nodes,max=$num_nodes,resource_properties="[\"=\", \"\$node_type\", \"$node_type\"]" \
  --start-date "$(date +'%Y-%m-%d %H:%M')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+6 hour')" \
  "$LEASE_NAME" \
  && wait_lease "$LEASE_NAME"

## A simple orchestration stack

For the first stack, we just aim to get you acquainted with what is possible. Take a look at the [single_server.yaml](./Orchestration/single_server.yaml) template. This is a _very_ simple template that just launches a single server. It is not very helpful on its own, and mostly serves to get you acquainted with the template file syntax. Orchestration templates on Chameleon use the [HOT syntax (Heat Orchestration Template)](https://docs.openstack.org/heat/rocky/template_guide/hot_guide.html), and you can find many details on the official OpenStack documentation, but here is a high-level overview of the sections:

- `description`: a brief text description of what the template creates
- `heat_template_version`: the schema version the template requires. Useful if you are using more advanced features of HOT. Please see the [HOT specification](https://docs.openstack.org/heat/latest/template_guide/hot_spec.html) for more information on what features are available in what versions.
- `parameters`: inputs that can (or must) be provided by the user to configure the stack. This allows you to for example adjust the size of a cluster provisioned by the template, but can also be used to pass information to server instances on boot to influence their startup behavior.
- `resources`: the resources to be provisioned. This is a dictionary where the keys are the names of the resource; resources are named so that they can refer to one another, which can be important for expressing dependencies between resources.
- `outputs`: any values to be exported by your stack when it is created. This is useful to capture dynamic properties, such as IP addresses, which are only known once the stack has been created.

You can use the `openstack stack create` command to launch an orchestration stack from a template file. Any parameters can be passed in via `--parameter` flags. The stack will initially be in the `CREATE_IN_PROGRESS` step, and should move to `CREATE_COMPLETE` once all resources are successfully provisioned.

In [ ]:
simple_stack="$LEASE_NAME-simple"

# We need to pass in a bare metal reservation for the `reservation_id` parameter.
# This reservation needs to have enough bare metal nodes reserved to satisfy the stack
# we are trying to launch. We can use the `lease_list_reservations` helper function for this.
reservation_id=$(jq -r 'map(select(.resource_type=="physical:host"))[0].id' <(lease_list_reservations "$LEASE_NAME"))

openstack stack create -t Orchestration/single_server.yaml \
  --parameter key_name="$KEYPAIR_NAME" \
  --parameter reservation_id="$reservation_id" \
  "$simple_stack"

#### Wait for stack creation to complete

Depending on how complex the stack is, it could take some time to create. In this case, we just need to wait for one instance to be provisioned, so the time is expected to be about ~10 minutes. You can use the `wait_stack` helper to pause execution until the stack has been created successfully. You can also watch the stack create via the [Stacks panel](https://chi.uc.chameleoncloud.org/project/stacks/) in the "Orchestration" section of the left-hand sidebar.

In [ ]:
wait_stack "$simple_stack"

### Query state of the orchestration

Now that your orchestration stack has been created, you can query various aspects of its state, if you like. This can be helpful if you are building some sort of automation around the orchestration. The following examples show you just a little bit of what is possible here.

In [ ]:
echo "All resources provisioned by stack '$simple_stack':"
echo
openstack stack resource list "$simple_stack" --max-width 80

# Get the ID of the "instance" resource
instance_id=$(openstack stack resource show "$simple_stack" "my_instance" -f value -c physical_resource_id)

echo
echo "Instance ID: $instance_id"
echo "Details:"
# Show details about the server instance
openstack server show "$instance_id" --max-width 80

echo
echo "Stack outputs:"
# List all outputs provided by the stack
openstack stack output show "$simple_stack" --all

# Get the IP address output
FLOATING_IP=$(openstack stack output show "$simple_stack" public_ip -f value -c output_value)

echo
echo "To log in: ssh cc@$FLOATING_IP"

### Connecting to the orchestration stack

The orchestration stack includes a Floating IP, which is bound to the server to allow public access. Recall, if you took a previous tutorial, how we had to set this up manually. One of the benefits of orchestration is being able to automate this away.

In [ ]:
wait_ssh "$FLOATING_IP" && ssh cc@$FLOATING_IP echo 'Connected to $(hostname)!'

### Clean up stack

Now that we're done, we can delete the stack, and also delete our lease, releasing these resources back in to the pool. Remember that your allocation is charged for the hours that your lease is active, so it's always a good thing to end the lease promptly when you are done with your reserved resources; the amount of time you did not use is immediately returned to your allocation: you are only charged for what you use!

In [ ]:
openstack stack delete "$simple_stack" --yes \
  && echo "Stack delete in progress."

## Hadoop cluster orchestration

The previous example was just an introduction into the very basics of the orchestration template. We haven't yet really explored the benefits of orchestration, namely, being able to provision a dynamic cluster of resources. We'll do that now! The next stack we will launch a Hadoop cluster. It will have a public IP address that you can use to connect via SSH and manage the rest of the nodes in the cluster. This is a nice way of efficiently using one of Chameleon's [Floating IP addresses](https://chameleoncloud.readthedocs.io/en/latest/technical/networks/networks_basic.html#floating-ip-addresses).

We will be using the [Hadoop Complex Appliance](https://www.chameleoncloud.org/appliances/53/) to save us a lot of work. Complex Appliances are orchestration templates written and maintained by Chameleon or users of Chameleon. Have a look at what Complex Appliances are available in the [Appliance Catalog](https://www.chameleoncloud.org/appliances/). If an appliance isn't available for the technology you need, it is usually possible to modify an existing appliance to support your needs, as in general many patterns are generalizable, e.g. a cluster of nodes, with some public IP address, with some isolated network.

In [ ]:
# We will name the stack the same as the lease for simplicity.
hadoop_stack="$LEASE_NAME-hadoop"
hadoop_network_name="$LEASE_NAME-network"

# We need to pass in a bare metal reservation for the `reservation_id` parameter.
# This reservation needs to have enough bare metal nodes reserved to satisfy the stack
# we are trying to launch. We can use the `lease_list_reservations` helper function for this.
reservation_id=$(jq -r 'map(select(.resource_type=="physical:host"))[0].id' <(lease_list_reservations "$LEASE_NAME"))

# Note that we subtract 1 from the $num_nodes - this is because the stack will provision
# a master node, plus $hadoop_worker_count nodes as additional nodes in the cluster.
openstack stack create -t <(curl -sS "https://www.chameleoncloud.org/appliances/api/appliances/53/template") \
  --parameter key_name="$KEYPAIR_NAME" \
  --parameter reservation_id="$reservation_id" \
  --parameter network_name="$hadoop_network_name" \
  --parameter hadoop_worker_count="$((num_nodes-1))" \
  "$hadoop_stack"

### Clean up cluster

In [ ]:
openstack stack delete "$hadoop_stack" --yes \
  && echo "Stack delete in progress."

### Clean up lease

Finally, if you're really done, you can also delete your lease.

In [ ]:
#blazar lease-delete "$LEASE_NAME"

## Recap

You (hopefully!) were able to succesfully provision both a simple orchestration stack and a more complex Hadoop cluster. You should now be a bit more familiar with what orchestration is, and know a tiny bit about the HOT syntax used by OpenStack Heat. Finally, you should have gotten a bit more knowledge about how to use the `openstack` CLI to drive orchestration on Chameleon.